In [3]:
%matplotlib inline
import itertools
from time import time
import csv


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from kifmm_py import KiFmm

# Single Precision

### BLAS Field Translations

In [4]:
# np.random.seed(0)
# surface_diff_vec = [0, 1, 2]
# svd_threshold_vec = [None, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 2e-2]
# depth_vec = [4, 5]
# expansion_order_vec = [3, 4, 5, 6]

# parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

# dim = 3
# dtype = np.float32
# ctype = np.complex64

# # Set FMM Parameters
# n_vec = 1
# n_crit = None
# depth = 3
# n_sources = 1000000
# n_targets = 1000000
# kernel = "laplace" 
# field_translation = "blas"  
# kernel_eval_type = (
#     "eval"  
# )

# sources = np.reshape(
#     np.random.rand(n_sources * dim), (n_sources, dim)
# ).astype(dtype)
# targets = np.reshape(
#     np.random.rand(n_targets * dim), (n_targets, dim)
# ).astype(dtype)
# charges = np.reshape(
#     np.random.rand(n_sources * n_vec), (n_sources, n_vec)
# ).astype(dtype)

# rel_error = []
# times = []
# setup = []

# for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
#     tmp = [expansion_order]*(depth+1)

#     s = time()
#     fmm = KiFmm(
#         tmp,
#         sources,
#         targets,
#         charges,
#         kernel_eval_type,
#         kernel,
#         field_translation,
#         prune_empty=True,
#         timed=True,
#         svd_threshold=svd_threshold,
#         surface_diff=surface_diff,
#         depth=depth
#     )
#     setup.append(time()-s)

#     s = time()
#     fmm.evaluate()
#     times.append(time()-s)
#     print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")
        
#     leaf = fmm.target_leaves[1]
#     found = fmm.potentials(leaf)[0]
#     targets_leaf = fmm.target_coordinates(leaf)
#     expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

#     relative_error = np.abs(expected-found)/expected

#     max_relative_error = np.max(relative_error)
#     min_relative_error = np.min(relative_error)
#     mean_relative_error = np.mean(relative_error)

#     rel_error.append((min_relative_error, mean_relative_error, max_relative_error))


# with open('accuracy_single_precision_blas.csv', mode='w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['surface_diff', 'svd_threshold', 'depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
#     for row in zip(parameters, times, rel_error):
#         writer.writerow([row[0][0], row[0][1], row[0][2], row[0][3], row[1], row[2][0], row[2][1], row[2][2]])

### FFT Field Translations

In [5]:
# np.random.seed(0)

# depth_vec = [4, 5]
# expansion_order_vec = [3, 4, 5, 6]

# parameters = list(itertools.product(depth_vec, expansion_order_vec))

# np.random.seed(0)

# dim = 3
# dtype = np.float32
# ctype = np.complex64

# # Set FMM Parameters
# n_vec = 1
# n_crit = None
# n_sources = 1000000
# n_targets = 1000000
# kernel = "laplace" 
# field_translation = "fft"  
# kernel_eval_type = (
#     "eval"  
# )

# sources = np.reshape(
#     np.random.rand(n_sources * dim), (n_sources, dim)
# ).astype(dtype)
# targets = np.reshape(
#     np.random.rand(n_targets * dim), (n_targets, dim)
# ).astype(dtype)
# charges = np.reshape(
#     np.random.rand(n_sources * n_vec), (n_sources, n_vec)
# ).astype(dtype)

# rel_error = []
# times = []
# setup = []


# for (i, (depth, expansion_order)) in enumerate(parameters):
#     tmp = [expansion_order]*(depth+1)
#     s = time()
#     fmm = KiFmm(
#         tmp,
#         sources,
#         targets,
#         charges,
#         kernel_eval_type,
#         kernel,
#         field_translation,
#         prune_empty=True,
#         timed=True,
#         depth=depth
#     )
#     setup.append(time()-s)

#     s = time()
#     fmm.evaluate()
#     times.append(time()-s)
#     print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")
        
#     leaf = fmm.target_leaves[1]
#     found = fmm.potentials(leaf)[0]
#     targets_leaf = fmm.target_coordinates(leaf)
#     expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

#     relative_error = np.abs(expected-found)/expected

#     max_relative_error = np.max(relative_error)
#     min_relative_error = np.min(relative_error)
#     mean_relative_error = np.mean(relative_error)

#     rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

# with open('accuracy_single_precision_fft.csv', mode='w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
#     for row in zip(parameters, times, rel_error):
#         writer.writerow([row[0][0], row[0][1], row[1], row[2][0], row[2][1], row[2][2]])

### Examine Parameters

In [6]:
blas = pd.read_csv('accuracy_single_precision_blas.csv')
fft = pd.read_csv('accuracy_single_precision_fft.csv')

In [26]:
# Filter for max rel error with given number of digits in final solution

fft_3 = fft[(fft['max_rel_err'] < 1e-3) & (fft['max_rel_err'] > 1e-4)]
fft_4 = fft[(fft['max_rel_err'] < 1e-4) & (fft['max_rel_err'] > 1e-5)]
fft_5 = fft[(fft['max_rel_err'] < 1e-5) & (fft['max_rel_err'] > 1e-6)]

blas_3 = blas[(blas['max_rel_err'] < 1e-3) & (blas['max_rel_err'] > 1e-4)]
blas_4 = blas[(blas['max_rel_err'] < 1e-4) & (blas['max_rel_err'] > 1e-5)]
blas_5 = blas[(blas['max_rel_err'] < 1e-5) & (blas['max_rel_err'] > 1e-6)]

In [30]:
blas_4

,surface_diff,svd_threshold,depth,expansion_order,time,min_rel_err,mean_rel_err,max_rel_err
2,0,NaN,4,5,0.579957,7.373443e-08,0.000005,0.000015
3,0,NaN,4,6,0.601354,7.146283e-08,0.000007,0.000022
6,0,NaN,5,5,0.544714,1.001439e-06,0.000007,0.000014
7,0,NaN,5,6,0.879691,2.696182e-06,0.000012,0.000020
10,0,1.000000e-07,4,5,0.571161,1.400418e-07,0.000005,0.000015
...,...,...,...,...,...,...,...,...
210,2,2.000000e-02,4,5,0.596778,0.000000e+00,0.000005,0.000014
211,2,2.000000e-02,4,6,0.614241,0.000000e+00,0.000005,0.000016
213,2,2.000000e-02,5,4,0.329107,5.418180e-07,0.000010,0.000040
214,2,2.000000e-02,5,5,0.376574,1.159918e-06,0.000008,0.000014


In [42]:
print("3 Digits")
print("-"*10)

min_index = blas_3['time'].idxmin()
print("----")
print("BLAS")
print("----")
print(blas_3.loc[min_index])
print("----")
print("FFT")
print("----")
min_index = fft_3['time'].idxmin()
print(fft_3.loc[min_index])

print()

print("4 Digits")
print("-"*10)
min_index = blas_4['time'].idxmin()
print("----")
print("BLAS")
print("----")
print(blas_4.loc[min_index])
print("----")
print("FFT")
print("----")
min_index = fft_4['time'].idxmin()
print(fft_4.loc[min_index])

print()

print("5 Digits")
print("-"*10)
min_index = blas_5['time'].idxmin()
print("----")
print("BLAS")
print("----")
print(blas_5.loc[min_index])

# print()
# min_index = fft_5['time'].idxmin()
# print(fft_5.loc[min_index])

3 Digits
----------
----
BLAS
----
surface_diff       0.000000
svd_threshold      0.010000
depth              5.000000
expansion_order    3.000000
time               0.211917
min_rel_err        0.000435
mean_rel_err       0.000457
max_rel_err        0.000505
Name: 52, dtype: float64
----
FFT
----
depth              5.000000
expansion_order    3.000000
time               0.187368
min_rel_err        0.000430
mean_rel_err       0.000451
max_rel_err        0.000499
Name: 4, dtype: float64

4 Digits
----------
----
BLAS
----
surface_diff       2.000000
svd_threshold      0.100000
depth              5.000000
expansion_order    4.000000
time               0.276603
min_rel_err        0.000008
mean_rel_err       0.000034
max_rel_err        0.000060
Name: 205, dtype: float64
----
FFT
----
depth              5.000000e+00
expansion_order    5.000000e+00
time               4.786139e-01
min_rel_err        4.635911e-07
mean_rel_err       6.755564e-06
max_rel_err        1.360971e-05
Name: 6, dtype: fl